In [1]:
import mlflow
import optax
import pandas as pd
import polars as pl
from flax import nnx
from nn_trainer.flax.loss_fn import mean_squared_error
from nn_trainer.flax.trainer import Trainer
from sklearn.model_selection import train_test_split

from nn_recsys.encoder import ColumnEncoder
from nn_recsys.evaluator import GeneralRatingEvaluator
from nn_recsys.loader import GeneralRatingLoader
from nn_recsys.model.FM import FM

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### READ

In [2]:
dataset_df = pl.from_pandas(
    pd.read_csv(
        "../../dataset/ML1M/ml-1m/ratings.dat",
        delimiter="::",
        engine="python",
        header=None,
        names=["user_id", "item_id", "rating", "timestamp"],
    )
)
display(dataset_df)

column_encoder = ColumnEncoder(
    user_id="user_id",
    item_id="item_id",
    rating="rating",
)

train_df, valid_df = train_test_split(
    dataset_df, test_size=0.1, random_state=0, shuffle=True
)

user_id,item_id,rating,timestamp
i64,i64,i64,i64
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
…,…,…,…
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746


In [3]:
categorical_X, numerical_X, y = column_encoder.fit_transform(train_df)
loader = GeneralRatingLoader(
    categorical_X=categorical_X,
    numerical_X=numerical_X,
    y=y,
    batch_size=512,
    rngs=nnx.Rngs(0),
)

categorical_X, numerical_X, y = column_encoder.transform(valid_df)
evaluator = GeneralRatingEvaluator(
    categorical_X=categorical_X, numerical_X=numerical_X, y=y
)

### 学習

In [4]:
model = FM(
    categorical_feature_cardinalities=list(column_encoder.cardinality_map.values()),
    numerical_feature_num=column_encoder.numerical_column_num,
    embed_dim=30,
    rngs=nnx.Rngs(0),
)

In [5]:
mlflow.set_tracking_uri(uri="http://localhost:8080")
mlflow.set_experiment("MF_RATING")

with mlflow.start_run() as run:
    trainer = Trainer(
        model=model,
        optimizer=optax.adamw(learning_rate=0.001, weight_decay=0.001),
        train_loader=loader,
        loss_fn=mean_squared_error,
        valid_evaluator=evaluator,
        early_stopping_patience=10,
        epoch_num=64,
        active_run=run,
    )
    trainer = trainer.fit()

[VALID 000]: loss=24.787647247314453, metrics={'mse': 24.787647247314453}


[TRAIN 001]: 100%|██████████| 1759/1759 [00:06<00:00, 271.17it/s, batch_loss=6.29]


[VALID 001]: loss=6.190262794494629, metrics={'mse': 6.190262794494629}


[TRAIN 002]: 100%|██████████| 1759/1759 [00:05<00:00, 311.13it/s, batch_loss=1.36]


[VALID 002]: loss=1.380133032798767, metrics={'mse': 1.380133032798767}


...

[VALID 017]: loss=0.8313101530075073, metrics={'mse': 0.8313101530075073}


[TRAIN 018]: 100%|██████████| 1759/1759 [00:06<00:00, 268.55it/s, batch_loss=0.56] 


[VALID 018]: loss=0.836018443107605, metrics={'mse': 0.836018443107605}


[TRAIN 019]: 100%|██████████| 1759/1759 [00:06<00:00, 271.35it/s, batch_loss=0.548]


[VALID 019]: loss=0.8411554098129272, metrics={'mse': 0.8411554098129272}
🏃 View run carefree-lark-998 at: http://localhost:8080/#/experiments/189505131723795868/runs/126150e345254290906fe1bcd5e5a339
🧪 View experiment at: http://localhost:8080/#/experiments/189505131723795868
